In [ ]:
import tensorflow.keras as keras

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

# <center>Introduction to deep learning<center>
    
## Data

For this lab we are going to use the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset.
This very standard dataset is widely used as a baseline in deep learning. Both Keras and Pytorch have datasets APIs containing this dataset.

In [ ]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### Task 1 - Vizualize the data:

In [ ]:
    YOURE CODE HERE

# A first model

We are going to use Keras' Sequential API

First we need to define the layers in a list:
```python
layers = [
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
]
```
<p>The `Flatten` layer transforms the matrix into a features vector.</p>
`Dense` layers (also known as fully connected) are classical matrix multiplications.

Then the model is defined as the sequence of layers, and compiled to define the loss and the optimizer
```python 
model = keras.Sequential(layers)

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
```

Finally we can train the network on the training dataset while testing on the test set
```python

history = model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test))
```

`history` stores the training curves.

### Task 2 - Implement and train the model on Mnist

In [ ]:
    YOUR CODE HERE

We can now plot the training curve to evaluate the quality of the training.

### Task 3 - Complete the `plot_history` to plot the train and test losses and accuracies curves.

In [ ]:
    YOUR CODE HERE

## Activation of the Units

To understand how a neural network interprete the data, we can visualize activation of the units.
In `Keras` a model can be defined only be specifying the input and the output if the weights to link them already exist. This is usefull to acces the inner layers of a network.
```ptyhon
intermediate_layer_model = keras.Model(inputs=model.input,
                                 outputs=model.layers[1].output)
```
The previous model takes as input an image and returns the activated units of the hidden layer.

In [ ]:
    YOUR CODE HERE

## Weights matrices

### Task 4 - Vizualize the weights connecting every pixel of the image to each units.

In [ ]:
    YOUR CODE HERE

# Convolutional network

With images, convolutinonal layers are more efficient than Dense layers. We need to add a 4th dimension to the input data to pass it to a convolution layer. This correspond to the channel, a standard RGB image as 3, but our gray image only has one.

Two new layers:
```python
keras.layers.Conv2D(8, kernel_size=3, activation="relu"),
keras.layers.MaxPool2D(pool_size=2, padding="valid"),
```
The convolution applies the convolution filters and the maxpooling layers aggregates the feature maps on squared regions.

### Task 5 - Implement a convolutional neural network with 2 convolutional layers and train it on MNIST.

#### Note :
Tensorflow does not dealocate the weights of a network, even if you delete the model. To completely clear the weights of a model you need to call `keras.backend.clear_session()`.


In [ ]:
    YOUR CODE HERE

### Task 6 - vizualize the weights of the first convolution layer

In [ ]:
    YOUR CODE HERE

### Task 7 - Using intermediate models, vizualize the outputs of the covolutions and max pooling for each layers.

In [ ]:
for layer in range(2):
    conv = keras.Model(inputs=model.input, outputs=model.layers[layer*2+1].output)
    pool = keras.Model(inputs=model.input, outputs=model.layers[layer*2+2].output)
    units = conv.predict(x_test)
    pooled = pool.predict(x_test)

    fig, axs = plt.subplots(1,8, figsize=(40,5))
    for _ in range(8):
        axs[_].imshow(np.array(units[1,:,:,_]), cmap="gray")
        axs[_].set_title("Laer %i - Unit map %i" % (layer, _+1))
        
    fig, axs = plt.subplots(1,8,figsize=(40,5))
    for _ in range(8):
        axs[_].imshow(np.array(pooled[1,:,:,_]), cmap="gray")
        axs[_].set_title("Laer %i - Unit map after pooling %i" % (layer, _+1))

    plt.show()

# Better optimization and Tensorboard

Up to now, we've let asside an important subject of deep learning, hyper-parameters tunning.

You can define an optimizer with the `optimizer` module of `Keras`:

```python
keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False)
keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False)
keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
```

One of the most usefull features of `tensorflow`, `tensorboard`, is extremely easy to set up in `Keras`.
```python
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
```

Another common practice is to decrease the learning rate when the network stops learning.
This can be achierved with a callback:
```python
lrate = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)
```

### Task 8 - Play with those optimizers to get more suitable results.

In [ ]:
    YOUR CODE HERE

You can now visualize your network and metrics with tensorboard. If you dont have it install it with ``` pip install tensorboard
``` and then run it with ```tensorboard --logdir logs```